In [32]:
import json
import spacy
import pickle
import itertools
from spacy.lang.en.stop_words import STOP_WORDS

In [2]:
with open("output.manifest") as fp:
    objects=[json.loads(object) for object in fp]

In [81]:
queries={}
for object in objects:
    for annotation in object['circumlocutions-all']['annotationsFromAllWorkers']:
            content=annotation['annotationData']['content']
            if len(content)>5 and len(content)<120 and nlp(object['source'])[0].vector_norm>.001 and ' ' not in  object['source'] and '/' not in  object['source']:
                queries[json.loads(content)['group']]=object['source']

In [88]:
num=len(queries)

In [4]:
nlp = spacy.load('/home/jcolbert/anaconda3/envs/lexical/lib/python3.6/site-packages/en_vectors_web_lg/en_vectors_web_lg-2.0.0')

In [13]:
test_words = pickle.load(open('../PSUTherapy/words.p','rb'))
test_words=list(itertools.chain.from_iterable([test_words[key] for key in test_words.keys()]))
test_tokens=[nlp(word)[0] for word in test_words]
test_tokens=[token for token in test_tokens if token.vector_norm>.0001]

In [40]:
for word in STOP_WORDS:
    for w in (word, word[0].capitalize(), word.upper()):
        lex = nlp_core.vocab[w]
        lex.is_stop = True

ValueError: cannot reshape array of size 0 into shape (684831,300)

In [41]:
query="A orange vegetable that bus bunny eats"

In [44]:
doc=nlp(query.lower())

In [64]:
get_scored_words(nlp(''.join([token.string for token in doc if not token.is_stop])))

[('orange', 0.7066666578209024),
 ('carrot', 0.6572613744763355),
 ('banana', 0.6528253537783654),
 ('strawberry', 0.6162683706802764),
 ('potato', 0.6160069796719414),
 ('egg', 0.6011352370200768),
 ('cheese', 0.5990258025809345),
 ('rabbit', 0.5958958816400495),
 ('tomato', 0.5938811926075166),
 ('cake', 0.5919060036909862),
 ('bread', 0.5902431328760251),
 ('butter', 0.5894558713447672),
 ('peanut', 0.5894504354389264),
 ('lemon', 0.5856990262666096),
 ('cow', 0.5774356073861284),
 ('goat', 0.5748330793260992),
 ('watermelon', 0.5685497426530849),
 ('pig', 0.5664280013109673),
 ('peach', 0.5647413120071644),
 ('onion', 0.5614212389330497),
 ('lime', 0.561320372961227),
 ('apple', 0.5581736447897863),
 ('sandwich', 0.5570775909540457),
 ('pineapple', 0.5558083410941002),
 ('coconut', 0.5486445338137325),
 ('corn', 0.5467439331848721),
 ('cat', 0.5444472521180241),
 ('pizza', 0.5376491084800256),
 ('crab', 0.532029059067422),
 ('baby', 0.5293441374352138),
 ('dog', 0.5283381029953442)

In [65]:
def get_scored_words(query):
    doc=nlp(query.lower())
    query=nlp(''.join([token.string for token in doc if not token.is_stop]))
    scores_map={token.text:query.similarity(token) for token in test_tokens}
    sorted_words = sorted(scores_map.items(), key=lambda kv: kv[1],reverse=True)
    return sorted_words

In [79]:
def get_rank(query):
    scored_words=get_scored_words(query[0])
    words=[scored_word[0] for scored_word in scored_words]
    return words.index(query[1])

In [82]:
ranks=[get_rank(query) for query in queries.items()]

In [92]:
len([i for i in ranks if i<10])/num

0.5562231759656653

In [93]:
10/450

0.022222222222222223

In [101]:
[ranks.index(max(ranks))]

TypeError: 'dict_items' object does not support indexing

In [120]:
import requests
from bs4 import BeautifulSoup

def get_google_text(query):
    global count
    if count % 20 == 0:
        print(count)
    count= count+1
    query=query.replace(' ','+')
    resp=requests.get("https://www.google.com/search?q=hopping+animal&oq="+query)
    soup=BeautifulSoup(resp.content)
    for script in soup(["script", "style"]):
        script.decompose()    # rip it out
    # get text
    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

In [115]:
queries_subset={}
count=0
for query in queries.items():
    if count % 10==0:
        queries_subset[query[0]]=query[1]
    count=count+1

In [116]:
len(queries_subset)

117

In [121]:
count=0
queries_google={query[0]:query[1] in get_google_text(query[0]) for query in queries.items()}

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160


In [122]:
queries_subset={}
count=0
for query in queries.items():
    if count % 10==0:
        queries_subset[query[0]]=query[1]
    count=count+1

In [123]:
count

1165

In [124]:
queries_google

{'an orange vegetable': False,
 'A orange vegetable that bus bunny eats': False,
 'human body part': False,
 'lower extremity of the body': True,
 'a soft head rest used in beds': False,
 'silverware item': False,
 'The band is the ... Spoon released their debut studio album, Telephono, in 1996 through Matador Records': False,
 'An edible fruit of a tree.': False,
 'variety of nut': False,
 'object used to magnify sound, usually voice': False,
 ' transducer': False,
 'communication device': False,
 'the part that connects the head with the body': False,
 'scientific instrument': False,
 'an optical instrument with a lens or combination of lenses used to enlarge images': False,
 'magnifying tool': False,
 'It is a sling made of fabric': False,
 'insect': False,
 'quicker is better.': False,
 'insect ': False,
 'something you mail letters in': False,
 'It is a common packaging item, usually made of thin, flat material': False,
 'A paper sleeve for notes and paper': False,
 'textile': Fal